In [1]:
%matplotlib inline

In [2]:
import numpy as np
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [21]:
class OVRLSVC():
    def __init__(self, **kwargs):
        self.c2svc = {}
        self.kwargs = kwargs
    
    def fit(self, X, y):
        for c in set(i for i in y):
            self.c2svc[c] = LinearSVC(**self.kwargs)
            self.c2svc[c].fit(X, y==c)
        return self
    
    def predict(self, X):
        confidences = np.zeros((X.shape[0], len(self.c2svc)))
        for c, svc in self.c2svc.items():
            confidences[:, int(c)] = svc.decision_function(X)
        result = confidences.argmax(axis=1)
        return result

    def score(self, X, y):
        result = self.predict(X)
        score = (result == y).sum() / len(y)
        return score

In [30]:
class OVRSVC():
    def __init__(self, **kwargs):
        self.c2svc = {}
        self.kwargs = kwargs
        
    def fit(self, X, y):
        for c in set(i for i in y):
            self.c2svc[c] = SVC(probability=True, kernel='linear', **self.kwargs)
            self.c2svc[c].fit(X, y==c)
        return self
    
    def predict(self, X):
        confidences = np.zeros((X.shape[0], len(self.c2svc)))
        for c, svc in self.c2svc.items():
            confidences[:, int(c)] = svc.predict_proba(X)[:, 1]
        result = confidences.argmax(axis=1)
        return result

    def score(self, X, y):
        result = self.predict(X)
        score = (result == y).sum() / len(y)
        return score

In [17]:
scaler = MinMaxScaler(copy=False)

X_train = np.load('data_hw2/train_data.npy')
y_train = np.load('data_hw2/train_label.npy').astype(int) + 1
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.2)
X_test = np.load('data_hw2/test_data.npy')
y_test = np.load('data_hw2/test_label.npy').astype(int) + 1

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_dev = scaler.transform(X_dev)
X_test = scaler.transform(X_test)

In [7]:
svc = SVC()
svc.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [8]:
print(svc.score(X_train, y_train))
print(svc.score(X_test, y_test))

0.911526212969
0.490874300854


In [18]:
lsvc = LinearSVC()
lsvc.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [31]:
print(lsvc.score(X_train, y_train))
print(lsvc.score(X_dev, y_dev))
print(lsvc.score(X_test, y_test))

1.0
1.0
0.467324109508


In [32]:
ovr_svc = OVRSVC()
ovr_svc.fit(X_train, y_train)

In [33]:
print(ovr_svc.score(X_dev, y_dev))
print(ovr_svc.score(X_test, y_test))

0.999464811346
0.48057109214


In [34]:
ovr_lsvc = OVRLSVC()
ovr_lsvc.fit(X_train, y_train)

In [35]:
print(ovr_lsvc.score(X_dev, y_dev))
print(ovr_lsvc.score(X_test, y_test))

1.0
0.467324109508
